In [1]:
import pandas as pd
from transformers import AutoTokenizer,AutoModel, BertTokenizer, BertModel
from datasets import load_dataset, Dataset
import nltk
from nltk.corpus import stopwords
import torch
import os

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kapsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import numpy as np

In [4]:
imdb = load_dataset('imdb',split='train')
imdb = imdb.shard(8, index=1)
imdb.set_format("torch",columns=['text','label'])

In [5]:
def remove_stop_words(example):
    wrds = example['text'].split(' ')
    flts = [w for w in wrds if w.lower() not in stop_words]
    str = ""
    
    for f in flts:
        str+= f+" "
    
    new_one = {'text':str[:-1],'label':example['label']}
    return new_one

In [6]:
print(type(imdb))

<class 'datasets.arrow_dataset.Dataset'>


In [7]:
imdb.map(remove_stop_words)

Dataset({
    features: ['text', 'label'],
    num_rows: 3125
})

In [8]:
imdbb = imdb.train_test_split(test_size=0.2,stratify_by_column='label')
imdb_train = imdbb['train']
imdb_test = imdbb['test']

In [9]:
TRAIN_BATCH_SIZE = 12
TEST_BATCH_SIZE = 4

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
n_train = imdb_train.shape[0] / TRAIN_BATCH_SIZE
tokenized_train_batches = []
tokenized_train_batches_y = []

for i in range(int(n_train)):
    btch = imdb_train['text'][TRAIN_BATCH_SIZE*i : min(TRAIN_BATCH_SIZE*(i+1), imdb_train.shape[0])]
    tp = tokenizer.batch_encode_plus(btch,max_length=512, padding='max_length', truncation=True,return_tensors='pt')
    tokenized_train_batches.append(tp)
    tokenized_train_batches_y.append(imdb_train['label'][TRAIN_BATCH_SIZE*i : min(TRAIN_BATCH_SIZE*(i+1), imdb_train.shape[0])])
    
    if i % (n_train//10) == 0:
        print(f' finished {int((i / (n_train)) * 100)}%')

print(len(tokenized_train_batches))

In [14]:
n_test = imdb_test.shape[0] / TEST_BATCH_SIZE

tokenized_test_batches = []
tokenized_test_batches_y = []

for i in range(int(n_test)):
    btch = imdb_test['text'][TEST_BATCH_SIZE*i : min(TEST_BATCH_SIZE*(i+1), imdb_test.shape[0])]
    tp = tokenizer.batch_encode_plus(btch,max_length=512, padding='max_length', truncation=True,return_tensors='pt')
    tokenized_test_batches.append(tp)
    tokenized_test_batches_y.append(imdb_test['label'][TEST_BATCH_SIZE*i : min(TEST_BATCH_SIZE*(i+1), imdb_test.shape[0])])
    
    if i % (n_test//10) == 0:
        print(f' finished {int((i / (n_test)) * 100)}%')

print(len(tokenized_test_batches))
print(len(tokenized_test_batches_y))

 finished 0%
 finished 9%
 finished 19%
 finished 28%
 finished 38%
 finished 48%
 finished 57%
 finished 67%
 finished 76%
 finished 86%
 finished 96%
156
156


In [4]:
class FBert(torch.nn.Module):
    
    def __init__(self) -> None:
        super(FBert,self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.2)
        self.l3 = torch.nn.Linear(768,64)
        self.l4 = torch.nn.Linear(64,1)
        self.l5 = torch.nn.Sigmoid()
        
    def forward(self, input):
        
        ids = input['input_ids']
        tto = input['token_type_ids']
        attn = input['attention_mask']
        
        _, t = self.l1(ids, attention_mask = attn, token_type_ids = tto, return_dict=False)
        # print(f'a1[0] -> {t.size()}')
        a2 = self.l2(t)
        # print(f'a2 -> {a2.size()}')
        a3 = self.l3(a2)
        # print(f' a3 -> {a3.size()}')
        a4 = self.l4(a3)
        # print(f' a4 -> {a4.size()}')
        a5 = self.l5(a4)
        # print(f' a5 -> {a5.size()}')
        a6 = a5.squeeze()
        # print(f' a6 -> {a6.size()}')
        return a6
        

In [7]:
my_model = FBert()
my_model = my_model.to(device)

In [20]:
loss_fn = torch.nn.BCELoss()
optim = torch.optim.Adam(my_model.parameters(),lr=1e-5)

In [ ]:
epochs = 10

In [36]:
test_loss = {}

In [ ]:
def train_one_epoch(epoch=0):
    my_model.train()
    
    count = 0
    
    for batch, batch_y in zip(tokenized_train_batches, tokenized_train_batches_y):
        batch.to(device)
        preds = my_model(batch)
        
        actual = torch.as_tensor(batch_y,device=device,dtype=torch.float)
        loss = loss_fn(preds,actual)

        optim.zero_grad()

        loss.backward()

        optim.step()
        
        if count % 20 == 0:
            print(f'onto batch: {count}')
            
        count += 1

In [38]:
def evaluate(model = my_model, epoch=0):
    
    loss_graph = []
    losses = []
    
    with torch.no_grad():
        
        my_model.eval()
        
        count = 0
        
        total_loss = 0
        
        for b, b_y in zip(tokenized_test_batches, tokenized_test_batches_y):
            b.to(device)
            test_pred = my_model(b)
            actual_test = torch.as_tensor(b_y,device=device,dtype=torch.float)
            ts_ls = loss_fn(test_pred, actual_test)
            losses.append(ts_ls)
            total_loss += ts_ls
            count +=1
            print(f'Batch {count}, Test loss: {total_loss/count}')
            loss_graph.append(total_loss/count)
            
        total_loss /= len(tokenized_test_batches)
        test_loss[epoch] = total_loss
        print(f'Epoch {epoch}, Test loss: {total_loss}')
        return total_loss, loss_graph, losses

In [16]:
def evaluate_preds(model, epoch=0):
    
    predicted = []
    actual = []
    
    with torch.no_grad():
        
        my_model.eval()
        
        for b, b_y in zip(tokenized_test_batches, tokenized_test_batches_y):
            b.to(device)
            test_pred = model(b)
            predicted.extend(test_pred.cpu().tolist())
            actual.extend(b_y.flatten().tolist())


        return predicted,actual

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_one_epoch()

In [ ]:
torch.save(my_model,'Models/fbert.pth')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
evaluate(model = my_model)

In [ ]:
torch.cuda.empty_cache()

In [24]:
p,a = evaluate_preds(my_model)

In [25]:
print(len(p),len(a))

624 624


In [28]:
print(type(p))
pclass = [1 if x > 0.5 else 0 for x in p]

<class 'list'>


In [30]:
set(pclass)

{0, 1}

In [31]:
def calculate_metrics(p,a):
    ps = precision_score(a,p)
    rs = recall_score(a,p)
    f1 = f1_score(a,p)
    ac = accuracy_score(a,p)
    
    return ps,rs,f1,ac

In [32]:
print(calculate_metrics(pclass,a))

(0.9201277955271565, 0.9230769230769231, 0.9216, 0.9214743589743589)
